<a href="https://colab.research.google.com/github/karnamohit/kranka_ucm/blob/master/tf1_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the useful libraries...

In [1]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
#from google.colab import drive

In [ ]:
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!ls /content/drive/My\ Drive

Checking the version of TensorFlow, NumPy...

In [2]:
print('TensorFlow version info:\t',tf.__version__)
#!pip show tensorflow
print(' ')
print('------------------------------------------------------------')
print(' ')
#!pip show numpy
print('NumPy version info:\t \t',np.__version__)

TensorFlow version info:	 1.14.0
 
------------------------------------------------------------
 
NumPy version info:	 	 1.16.4


**Making, processing raw data**

Make up data (2x2, sequentially indexed matrices)...

In [3]:
series_data = np.zeros((100,2,2), np.float64)        # "time"-series data for a 2x2 evolving matrix with 100 time-steps, randomly initialized
for i in range(series_data.shape[0]):
    series_data[i,:,:] = (i**0.5)*3 + 5
#print(series_data)
tsteps = int(series_data.shape[0])         # total number of time-steps in the series
tsteps_train = int(tsteps/2)               # number of time-steps used for training
mat_row = int(series_data.shape[1])
mat_col = int(series_data.shape[2])

#print(series_data[tsteps_train-2,:,:])
#print(series_data[tsteps_train-1,:,:])
#print(series_data[tsteps_train,:,:])

Define a sequence (multiple consecutive 2x2 matrices) size, ```len_seq```, to be fed in as input (equivalent to the amount of memory in previous time-steps, in our case), and the amount of overlap, ```n_overlap``` between any two adjacent input sequences...

In [44]:
n_pred = 1 # number of steps to be predicted
len_seq = 3 # length of input-sequence
n_overlap = 2 # overlap window between consecutive sequences
n_seq = int((tsteps_train - (len_seq - n_overlap)) / (len_seq - n_overlap)) # number of sequences of size len_seq

if (n_overlap >= len_seq):
    raise Exception("n_overlap must be less than len_seq")
else:
    if (n_pred >= (tsteps_train - (len_seq - 1))):
        raise Exception("n_pred must be MUCH less than the difference b/w the total time-steps and the length of the input sequence")

Build tensor of sequences, call it ```seq_series_data```...

In [57]:
seq_series_data = np.zeros((n_seq, len_seq, mat_row, mat_col), np.float64)
seq_series_data_pred = np.zeros((n_seq, n_pred, mat_row, mat_col), np.float64)

i = 0
k = 0

while (i < tsteps and k < n_seq):       # the training sequences start at time-step 1 (series_data[0] element) in this case
    try:
        for j in range(len_seq):
            seq_series_data[k,j,:,:] = series_data[(i+j),:,:]
            seq_series_data_pred[k,0:(1+n_pred),:,:] = series_data[(i+j+1):(i+j+n_pred+1),:,:]
    except IndexError:
        raise Exception("ummm... Houston, we've had a problem.")
    #seq_series_data_pred = np.append(seq_series_data_pred, series_data[(i+j+1):(i+j+1+n_pred),:,:], axis = 1)
    i += len_seq - n_overlap
    k += 1
    #print(k,j,i)
    #print(seq_series_data[k,:,:,:])

#seq_series_data_pred = np.delete(seq_series_data_pred, seq_series_data_pred[0,:,:,:], axis=0) # true output

x_inp = np.reshape(seq_series_data[:,:len_seq,:,:], [n_seq, len_seq, (mat_row*mat_col)])
y_tru = np.reshape(seq_series_data_pred[:,:,:,:], [n_seq, n_pred, (mat_row*mat_col)])


#print(seq_series_data_pred)
#print(type(seq_series_data))
#print(n_seq)
print(seq_series_data.shape)
print(seq_series_data_pred.shape)
#print(seq_series_data_pred[-1,:,:,:])
print(x_inp.shape)
print(y_tru.shape)

(49, 3, 2, 2)
(49, 1, 2, 2)
(49, 3, 4)
(49, 1, 4)


In [ ]:
# test to check data-type and shape of seq_series_data, seq_series_data_pred, and a slice seq_series_data

print('shape of X (excluding the true output): ',seq_series_data[0,:-1,:,:].shape) 
print('no. of batches of X: ', seq_series_data.shape[0]) # 1 - prints the number of time-steps, each of these associated with a sequence of 2x2 matrices
#print(seq_series_data[0].shape)
print('shape of Y_pred and Y_true: ', seq_series_data_pred[:1,:,:].shape)
print('no. of predictions (equal to the no. of batches of X): ', seq_series_data_pred.shape[0]) 
            # 2 - prints the number of time-steps, each of these assocaited with one 2x2 matrix (model truth), must match 1 above
print('shape of X (excluding the true output): ', seq_series_data[0,:-1,:,:].shape) # prints the sequence-size used per prediction

shape of X (excluding the true output):  (3, 2, 2)
no. of batches of X:  49
shape of Y_pred and Y_true:  (1, 2, 2)
no. of predictions (equal to the no. of batches of X):  49
shape of X (excluding the true output):  (3, 2, 2)


**Building a model**

In [58]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Define activation functions, operations, etc. ...

In [59]:
# using "activation=None" (linear operation) in tf.layers.dense for hidden layer h_1

def softplus(z, name="softplus"):
    return tf.math.softplus(z)

def exponential(z, name="exponential"):
    return tf.math.exp(z)

def square(z, name="square"):
    return tf.math.square(z)

def expinv(z, name="expinv"):
    return tf.math.exp(-z)

def inv(z, name="inv"):
    return tf.math.square(tf.math.sqrt(1/z))

'''# using a convolution operation (with a custom filter), conv_3D, for hidden layer h_2

# define a custom filter, filter_1, to be used with the one-shot (stride-less) convolution operation, conv_3D_oneshot
#     ignoring padding for now
def filter_1(d, h, w):
    # the latest sequence gets the highest contribution
    filter = np.zeros((d, h, w), np.float64)
    for i in range(d):
        for j in range(h):
            for k in range(w):
                filter[i,j,k] = np.sqrt(np.exp(d))
    return tf.convert_to_tensor(filter, dtype=tf.int32)

# define a (stride-less) function for the convolution operation
def conv_3D_oneshot(z, name="custom_conv_3D_1"):
    shape = z.get_shape().as_list()
    depth = shape[0].value
    height = shape[1].value
    width = shape[2].value
    filter = filter_1(depth, height, width)
    return tf.math.reduce_sum(tf.math.multiply(z, filter), 0)'''

'# using a convolution operation (with a custom filter), conv_3D, for hidden layer h_2\n\n# define a custom filter, filter_1, to be used with the one-shot (stride-less) convolution operation, conv_3D_oneshot\n#     ignoring padding for now\ndef filter_1(d, h, w):\n    # the latest sequence gets the highest contribution\n    filter = np.zeros((d, h, w), np.float64)\n    for i in range(d):\n        for j in range(h):\n            for k in range(w):\n                filter[i,j,k] = np.sqrt(np.exp(d))\n    return tf.convert_to_tensor(filter, dtype=tf.int32)\n\n# define a (stride-less) function for the convolution operation\ndef conv_3D_oneshot(z, name="custom_conv_3D_1"):\n    shape = z.get_shape().as_list()\n    depth = shape[0].value\n    height = shape[1].value\n    width = shape[2].value\n    filter = filter_1(depth, height, width)\n    return tf.math.reduce_sum(tf.math.multiply(z, filter), 0)'

Define architectural (hyper-)parameters...

In [135]:
# define layers with #(nodes/layer), n_units_i, connected by weight matrices, w_i, and bias matrices, b_i

n_inp_n = seq_series_data.shape[0]   # no. of 2x2 matrices fed as input ("units" per INPUT layer)
n_units_1 = 16   # no. of units in the first hidden layer
#n_units_2 = 8
#n_units_3 = 4
n_units_1_conv = n_units_1
            # no. of (convolution) units in the output layer; must equal the number of units in the immediately preceding hidden layer
n_outp_n  = n_pred
            # number of (2x2) tensors the model must output (the same size as the sequence-size of the model output, 1 in the current case)

In [136]:
n_inp_n

49

Construct input placeholders for the model, build the layers...

In [150]:
reset_graph()

X = tf.placeholder(tf.float64, shape=(None, len_seq, (mat_row*mat_col)), name='input')    # raw input (?x3x4)
'''
Y_pred = tf.placeholder(tf.float64, shape=(None, mat_row, mat_col),name='model_output')
            # model output: predicting a 2x2 matrix "?" time-step(s) at a time
'''
Y_true = tf.placeholder(tf.float64, shape=(None, n_pred, (mat_row*mat_col)), name='true_output')
      # true output: same shape as the model output

with tf.name_scope("ann_1"):    # test ANN model, "ann_1"
    h_1 = tf.layers.dense(X, n_units_1, activation=tf.nn.relu, use_bias=True, name="hidden_layer_1")
          # first hidden layer (linear activation)
    #h_2 = tf.layers.dense(h_1, n_units_2, activation=None, use_bias=False, name="hidden_layer_2")
    #h_3 = tf.layers.dense(h_2, n_units_3, activation=None, use_bias=False, name="hidden_layer_3")
    f_1 = tf.get_variable("inp_to_outp_1", [n_units_1, n_outp_n], trainable=True, dtype=tf.float64)
          # "filter" with zero strides, for "one-shot convolution"
    hf_1 = tf.squeeze(tf.tensordot(h_1, f_1, axes = 1, name="hidden_funnel_layer_1"), 2)
          # funneling layer, to reduce input dimensions to output dimensions 
    f_o = tf.get_variable("hf_to_outp_1", [len_seq, n_outp_n, 1], trainable=True, dtype=tf.float64)
    hf_o = tf.tensordot(hf_1, f_o, axes = 1, name="output_funnel_layer")    # output funnel layer
    Y_pred = tf.layers.dense(hf_o, (mat_row*mat_col), activation=tf.nn.elu, use_bias=True, name="output_layer")    # model output

In [142]:
print(Y_pred.shape)

(?, 1, 4)


Define the loss function; choose the optimization algorithm...

In [152]:
with tf.name_scope("loss_ann_1"):   # loss function for the model "ann_1"
    loss = tf.losses.mean_squared_error(Y_pred, Y_true)   # element-wise MSE

learn_rate = 0.001    # hyper-parameter for updating a variable v: v_new = v_old + delta_v = v_old - learn_rate*grad_v(loss)

with tf.name_scope("opt_ann_1"):    # optimizer for "ann_1"
    opt = tf.train.GradientDescentOptimizer(learn_rate)   # use the gradient descent optimization algorithm
    loss_min = opt.minimize(loss)   # operation to minimize "loss"

**Running the model**

In [144]:
'''print(series_data.shape)
inp = np.expand_dims(series_data[:99,:,:],-1)
out = series_data[1:,:,:]
print(inp.shape)
print(out.shape)'''

'print(series_data.shape)\ninp = np.expand_dims(series_data[:99,:,:],-1)\nout = series_data[1:,:,:]\nprint(inp.shape)\nprint(out.shape)'

Assign no. of training cycles, batch-sizes, etc.; set up initializer and session-saver objects ```init``` and ```saver``` (required for assigning initial values to the ```tf.Graph``` variables and saving the values of variables optimized in the session); use a context manager to run the ```tf.Session```...

In [153]:
n_epoch = 20000   # number of optimization steps on the WHOLE training dataset

init = tf.global_variables_initializer()    # initialize the model variables
saver = tf.train.Saver()    # saves the updated parameters from a session run

with tf.Session() as sess:
    init.run()
    """#inp = np.expand_dims(series_data[:n_inp_n,:,:],-1)
    #out = np.expand_dims(series_data[n_inp_n,:,:],0)
    #n_outp_n = 3
    n_len = len(series_data)
    #print(n_len,n_outp_n)
    inp = np.expand_dims(series_data[:(n_len - n_outp_n),:,:],-1)
    out = np.reshape(series_data[(n_len - n_outp_n):,:,:], [1,mat_row,mat_col,n_outp_n])
    print(inp.shape)
    print(out.shape)
    """
    y_out = Y_pred.eval(feed_dict={X: x_inp})
    #print(y_out)
    for train_epoch in range(n_epoch):
        #for batch_iter in range(seq_series_data.shape[0]):
        #print(inp.shape, out.shape)
        #print(X_in.get_shape(),Y_pred.get_shape(), Y_true.get_shape(), hf_1.get_shape(), f_1.get_shape(), h_1.get_shape())
        sess.run(loss_min, feed_dict={X: x_inp, Y_true: y_tru})
        if (train_epoch % 1000 == 0):
            y_out = Y_pred.eval(feed_dict={X: x_inp})
            #layer_2 = hf_1.eval(feed_dict={X: inp})
            #layer_3 = h_2.eval(feed_dict={X: inp})
            acc_train = loss.eval(feed_dict={X: x_inp, Y_true: y_tru})
            print(train_epoch, "MSE (training):", acc_train, "\n", y_out.shape)#, "\n", layer_2, "\n", layer_3)

    vars = tf.trainable_variables()
    for v in vars:
        print(v)
        print(sess.run(v))

0 MSE (training): 278.49365 
 (49, 1, 4)
1000 MSE (training): 235.66342 
 (49, 1, 4)
2000 MSE (training): 99.48665 
 (49, 1, 4)
3000 MSE (training): 49.402534 
 (49, 1, 4)
4000 MSE (training): 30.982227 
 (49, 1, 4)
5000 MSE (training): 24.207468 
 (49, 1, 4)
6000 MSE (training): 21.715797 
 (49, 1, 4)
7000 MSE (training): 20.799393 
 (49, 1, 4)
8000 MSE (training): 20.46235 
 (49, 1, 4)
9000 MSE (training): 20.338388 
 (49, 1, 4)
10000 MSE (training): 20.292797 
 (49, 1, 4)
11000 MSE (training): 20.276031 
 (49, 1, 4)
12000 MSE (training): 20.269865 
 (49, 1, 4)
13000 MSE (training): 20.267597 
 (49, 1, 4)
14000 MSE (training): 20.266764 
 (49, 1, 4)
15000 MSE (training): 20.266455 
 (49, 1, 4)
16000 MSE (training): 20.266342 
 (49, 1, 4)
17000 MSE (training): 20.266302 
 (49, 1, 4)
18000 MSE (training): 20.266283 
 (49, 1, 4)
19000 MSE (training): 20.266281 
 (49, 1, 4)
<tf.Variable 'hidden_layer_1/kernel:0' shape=(4, 16) dtype=float64_ref>
[[-0.39961986 -0.39992093 -0.46756312 -0.26